# Hazard assessment for river flooding using river discharge statistics
## Accessing data

This notebook illustrates how the river discharges dataset can be downloaded via API from the Copernicus Data Store for subsequent use in the analysis. The dataset is downloaded for the entire Europe, it is not possible to subset it by area prior to downloading. 

Note: alternatively it is possible to access this dataset via the dataset mirror on the CLIMAAX data server, this option is made available to speed up data access. 

### Load libraries

`````{admonition} Find more info about the libraries used in this workflow here
:class: hint dropdown

In this notebook we will use the following Python libraries:
- [os](https://docs.python.org/3/library/os.html) - Provides a way to interact with the operating system, allowing the creation of directories and file manipulation.
- [xarray](https://docs.xarray.dev/en/stable/) - library for working with labelled multi-dimensional arrays.
- cdsapi
- zipfile

These libraries enable the download of the dataset.
`````

In [2]:
import os
from glob import glob
import cdsapi
import zipfile
import xarray as xr

### Create the directory structure
In the next cell will create the directory called 'FLOOD_RIVER_discharges' in the same directory where this notebook is saved. A folder for storing data will be made as well.

In [3]:
# Define the folder for the flood workflow
workflow_folder = 'FLOOD_RIVER_discharges'
os.makedirs(workflow_folder, exist_ok=True)

data_folder = os.path.join(workflow_folder, 'data')
os.makedirs(data_folder, exist_ok=True)

data_folder_catch = os.path.join(data_folder, 'EHYPEcatch')
os.makedirs(data_folder_catch, exist_ok=True)

### Data access parameters

In the cell below we will select three GCM-RCM model combinations (see dataset documentation for the available combinations). Using several model combinations helps to assess the uncertainty range due to the different climate models in the river discharges data.  

In [4]:
gcms = ["ec_earth","hadgem2_es","mpi_esm_lr"]
rcms = ["racmo22e","rca4","csc_remo2009"]
ens_members = ['r12i1p1','r1i1p1','r1i1p1']

We also need to initialize the API client to be able to make connection to the CDS servers for downloading the data.

In [4]:
client = cdsapi.Client()

2025-08-18 17:27:18,577 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


### Downloading river discharge timeseries - historical daily values

First we will download catchment-level discharge data for the historical period. Data is available based on different E-HYPEcatch model realizations. We will download all model realizations.

The daily timeseries are downloaded for the period of 2000-2005. If a different period is required for comparing to local observations, the selection can be adjusted below as part of the API request under "period".

In [ ]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "essential_climate_variables",
        "variable": ["river_discharge"],
        "variable_type": "absolute_values",
        "time_aggregation": "daily",
        "experiment": ["historical"],
        "hydrological_model":   ["e_hypecatch_m00",
                                "e_hypecatch_m01",
                                "e_hypecatch_m02",
                                "e_hypecatch_m03",
                                "e_hypecatch_m04",
                                "e_hypecatch_m05",
                                "e_hypecatch_m06",
                                "e_hypecatch_m07"],
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["2001_2005"]
    }
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)

2025-07-23 16:28:30,401 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-07-23 16:28:30,402 INFO Request ID is 8ded3f52-88c2-494c-97e7-928cddba03ae
2025-07-23 16:28:30,538 INFO status has been updated to accepted
2025-07-23 16:29:20,362 INFO status has been updated to running
2025-07-23 16:42:49,984 INFO status has been updated to successful
2025-07-23 16:43:34,731 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-07-23 16:43:34,732 INFO Request ID is a4f7230d-5855-427e-9435-e8a8e6d1c1cc
2025-07-23 16:43:34,810 INFO status has been updated to accepted
2025-07-23 16:43:

In [31]:
def preprocess_daily(ds):
    filename = ds.encoding['source'].split("/")[-1].split("\\")[-1]
    ds['gcm_rcm'] = f'{filename.split("_")[3]}_{filename.split("_")[6]}'
    ds = ds.set_coords('gcm_rcm').expand_dims('gcm_rcm')

    ds['catchmodel'] = filename.split("_")[2]
    ds = ds.set_coords('catchmodel').expand_dims('catchmodel')    

    return ds

In [32]:
# CHECK
files = glob(os.path.join(data_folder_catch, 'rdis_day_E-HYPEcatch*-EUR-11_*_catch_v1.nc'))
ds_day = xr.open_mfdataset(files, preprocess=preprocess_daily)

In [33]:
ds_day

<xarray.Dataset> Size: 6GB
Dimensions:     (catchmodel: 8, gcm_rcm: 3, time: 1826, id: 34810)
Coordinates:
  * time        (time) datetime64[ns] 15kB 2001-01-01 2001-01-02 ... 2005-12-31
  * id          (id) int32 139kB 8801654 8000123 8212459 ... 9605711 9601936
  * gcm_rcm     (gcm_rcm) <U33 396B 'ICHEC-EC-EARTH_KNMI-RACMO22E' ... 'MPI-M...
  * catchmodel  (catchmodel) <U20 640B 'E-HYPEcatch00-EUR-11' ... 'E-HYPEcatc...
Data variables:
    rdis        (catchmodel, gcm_rcm, time, id) float32 6GB dask.array<chunksize=(1, 1, 1, 34810), meta=np.ndarray>
Attributes: (12/24)
    CDI:                      Climate Data Interface version 1.9.8 (https://m...
    history:                  Wed Mar 31 15:57:59 2021: cdo selyear,2001,2002...
    source:                   A set of EURO-CORDEX EUR-11 RCMS were bias adju...
    institution:              SMHI, www.smhi.se
    Conventions:              CF-1.6
    NCO:                      4.4.8
    ...                       ...
    invar_experiment_name:    rcp45
    invar_rcm_model_id:       KNMI-RACMO22E
    time_period:              2001-2010
    contact:                  copernicus-support@ecmwf.int
    data_quality:             C3S_424_SMHI uses state of the art, quality con...
    CDO:                      Climate Data Operators version 1.9.8 (https://m...

In [34]:
ds_day.to_netcdf(os.path.join(data_folder, 'rdis_day_E-HYPEcatch_allmodels.nc'))

### Downloading river discharge timeseries - monthly means

Next we will download the historical monthly means of river discharges for 1971-2000 from the E-HYPEcatch models which are useful for checking longer-term statistics of river discharges in the historical climate.

In [ ]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "climate_impact_indicators",
        "variable": ["river_discharge"],
        "variable_type": "absolute_values",
        "time_aggregation": "monthly_mean",
        "experiment": ["historical"],
        "hydrological_model":   ["e_hypecatch_m00",
                                "e_hypecatch_m01",
                                "e_hypecatch_m02",
                                "e_hypecatch_m03",
                                "e_hypecatch_m04",
                                "e_hypecatch_m05",
                                "e_hypecatch_m06",
                                "e_hypecatch_m07"],
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["1971_2000"]
    }
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)


We will download monthly means of discharges for future periods of 2011-2040, 2041-2070 and 2071-2100:

In [ ]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]

    for period in ["2011_2040","2041_2070","2071_2100"]:
        file = os.path.join(data_folder_catch, 'download.zip')
        dataset = "sis-hydrology-variables-derived-projections"
        request = {
            "product_type": "climate_impact_indicators",
            "variable": ["river_discharge"],
            "variable_type": "absolute_values",
            "time_aggregation": "monthly_mean",
            "experiment": ["rcp_4_5","rcp_8_5"],
            "hydrological_model":  ["e_hypecatch_m00",
                                    "e_hypecatch_m01",
                                    "e_hypecatch_m02",
                                    "e_hypecatch_m03",
                                    "e_hypecatch_m04",
                                    "e_hypecatch_m05",
                                    "e_hypecatch_m06",
                                    "e_hypecatch_m07"],
            "rcm": rcm,
            "gcm": gcm,
            "ensemble_member": ens_member,
            "period": period
        }
        client.retrieve(dataset, request, file)

        # Unzip the file that was just downloaded, and remove the zip file
        with zipfile.ZipFile(file, 'r') as zObject:
            zObject.extractall(path=data_folder_catch)
        os.remove(file)

We will make use of a preprocessing function to write model names and scenarios to the dataset dimensions:

In [6]:
def preprocess_monthly_mean(ds):
    filename = ds.encoding['source'].split("/")[-1].split("\\")[-1]
    ds['gcm_rcm'] = f'{filename.split("_")[4]}_{filename.split("_")[7]}'
    ds = ds.set_coords('gcm_rcm').expand_dims('gcm_rcm')

    ds['catchmodel'] = filename.split("_")[3]
    ds = ds.set_coords('catchmodel').expand_dims('catchmodel')    

    ds['scenarios'] = filename.split("_")[5]
    ds = ds.set_coords('scenarios').expand_dims('scenarios')

    ds['time_period'] = filename.split("_")[9]
    ds = ds.set_coords('time_period').expand_dims('time_period')

    ds['time'] = ds.time.dt.month

    return ds

Now we can read the dataset of monthly means of river discharges into a single dataset variable and save it on disk as one file for ease of future access.

In [10]:
files = glob(os.path.join(data_folder_catch, 'rdis_ymonmean_abs_E-HYPEcatch*-EUR-11_*_na_*_catch_v1.nc'))
ds_monmean = xr.open_mfdataset(files, preprocess=preprocess_monthly_mean)

In [11]:
ds_monmean

<xarray.Dataset> Size: 642MB
Dimensions:        (time_period: 4, scenarios: 3, catchmodel: 8, gcm_rcm: 4,
                    time: 12, id: 34810)
Coordinates:
  * time           (time) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * id             (id) int32 139kB 8801654 8000123 8212459 ... 9605711 9601936
  * catchmodel     (catchmodel) <U20 640B 'E-HYPEcatch00-EUR-11' ... 'E-HYPEc...
  * time_period    (time_period) <U9 144B '1971-2000' ... '2071-2100'
  * scenarios      (scenarios) <U10 120B 'historical' 'rcp45' 'rcp85'
  * gcm_rcm        (gcm_rcm) <U36 576B 'ICHEC-EC-EARTH_KNMI-RACMO22E-v1' ... ...
Data variables:
    rdis_ymonmean  (time_period, scenarios, catchmodel, gcm_rcm, time, id) float32 642MB dask.array<chunksize=(2, 1, 1, 1, 1, 34810), meta=np.ndarray>
Attributes: (12/28)
    CDI:                      Climate Data Interface version 1.9.5 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    comment:                  -
    CDO:                      Climate Data Operators version 1.9.5 (http://mp...
    history:                  CDO commands (last cdo command first and separa...
    ...                       ...
    invar_hm_model_id:        Hydrological models in the order of the variabl...
    invar_experiment_name:    rcp45
    time_coverage_start:      19710101
    time_coverage_end:        20001231
    variable_name:            rdis_ymonmean
    contact:                  copernicus-support@ecmwf.int

In [12]:
ds_monmean.to_netcdf(os.path.join(data_folder, 'rdis_ymonmean_abs_E-HYPEcatch_allmodels.nc'))

### Downloading data on flood occurence (extreme river discharges)

We will download river discharge data corresponding to the 10-year and 50-year return periods (extreme river discharges projected to be exceeded once in 10 years and once in 50 years). Similarly to the timeseries data, we will download this data for different climate scenarios, timelines and catchment models.

Downloading 10-year and 50-year return period river discharges for the historical climate:

In [5]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]
    
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "climate_impact_indicators",
        "variable": ["flood_recurrence_10_years_return_period",
                        "flood_recurrence_50_years_return_period"],
        "variable_type": "absolute_values",
        "time_aggregation": "annual_mean",
        "experiment": ["historical"],
        "hydrological_model": ["e_hypecatch_m00",
                                "e_hypecatch_m01",
                                "e_hypecatch_m02",
                                "e_hypecatch_m03",
                                "e_hypecatch_m04",
                                "e_hypecatch_m05",
                                "e_hypecatch_m06",
                                "e_hypecatch_m07"],
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["1971_2000"]
    }   
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)

2025-08-18 17:27:28,964 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 17:27:28,965 INFO Request ID is c0d0b0b6-4c85-4cb8-926f-0abe44218d61
2025-08-18 17:27:29,048 INFO status has been updated to accepted
2025-08-18 17:27:42,626 INFO status has been updated to successful
2025-08-18 17:27:44,076 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 17:27:44,077 INFO Request ID is 68a825d8-fa3c-4107-95fd-71d5a35ae8be
2025-08-18 17:27:44,126 INFO status has been updated to accepted
2025-08-18 17:27:57,630 INFO status has been updated to successful
2025-08-18 17:

Downloading 10-year and 50-year return period river discharges for the future time periods in terms of absolute values:

In [6]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]

    for period in ["2011_2040","2041_2070","2071_2100"]:
        file = os.path.join(data_folder_catch, 'download.zip')
        dataset = "sis-hydrology-variables-derived-projections"
        request = {
            "product_type": "climate_impact_indicators",
            "variable": ["flood_recurrence_10_years_return_period",
                         "flood_recurrence_50_years_return_period"],
            "variable_type": "absolute_values",
            "time_aggregation": "annual_mean",
            "experiment": ["rcp_4_5","rcp_8_5"],
            "hydrological_model":  ["e_hypecatch_m00",
                                    "e_hypecatch_m01",
                                    "e_hypecatch_m02",
                                    "e_hypecatch_m03",
                                    "e_hypecatch_m04",
                                    "e_hypecatch_m05",
                                    "e_hypecatch_m06",
                                    "e_hypecatch_m07"],
            "rcm": rcm,
            "gcm": gcm,
            "ensemble_member": ens_member,
            "period": period
        }   
        client.retrieve(dataset, request, file)

        # Unzip the file that was just downloaded, and remove the zip file
        with zipfile.ZipFile(file, 'r') as zObject:
            zObject.extractall(path=data_folder_catch)
        os.remove(file)

2025-08-18 17:28:21,339 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 17:28:21,340 INFO Request ID is 742d6d04-ae76-471c-b85d-f94932058f4f
2025-08-18 17:28:21,395 INFO status has been updated to accepted
2025-08-18 17:28:29,748 INFO status has been updated to successful
2025-08-18 17:28:31,173 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 17:28:31,174 INFO Request ID is bcf631e4-207a-434b-9b6f-390d33571342
2025-08-18 17:28:31,249 INFO status has been updated to accepted
2025-08-18 17:28:39,569 INFO status has been updated to running
2025-08-18 17:28:

Downloading 50-year return period river discharges for the future time periods as relative values:

In [43]:
for ii, rcm in enumerate(rcms):
    gcm = gcms[ii]
    ens_member = ens_members[ii]

    for period in ["2011_2040","2041_2070","2071_2100"]:
        file = os.path.join(data_folder_catch, 'download.zip')
        dataset = "sis-hydrology-variables-derived-projections"
        request = {
            "product_type": "climate_impact_indicators",
            "variable": ["flood_recurrence_10_years_return_period",
                         "flood_recurrence_50_years_return_period"],
            "variable_type": "relative_change_from_reference_period",
            "time_aggregation": "annual_mean",
            "experiment": ["rcp_4_5","rcp_8_5"],
            "hydrological_model":  ["e_hypecatch_m00",
                                    "e_hypecatch_m01",
                                    "e_hypecatch_m02",
                                    "e_hypecatch_m03",
                                    "e_hypecatch_m04",
                                    "e_hypecatch_m05",
                                    "e_hypecatch_m06",
                                    "e_hypecatch_m07"],
            "rcm": rcm,
            "gcm": gcm,
            "ensemble_member": ens_member,
            "period": period
        }   
        client.retrieve(dataset, request, file)

        # Unzip the file that was just downloaded, and remove the zip file
        with zipfile.ZipFile(file, 'r') as zObject:
            zObject.extractall(path=data_folder_catch)
        os.remove(file)

2025-08-18 16:50:58,285 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 16:50:58,286 INFO Request ID is c530b41f-5a2a-4baf-b76f-e4fa7c88aea8
2025-08-18 16:50:58,358 INFO status has been updated to accepted
2025-08-18 16:51:12,002 INFO status has been updated to running
2025-08-18 16:51:19,677 INFO status has been updated to successful
2025-08-18 16:51:20,975 INFO [2025-01-29T00:00:00] This dataset is no longer supported by the data providers. Data and documentation are provided as is. Users are encouraged to use our [Forum](https://forum.ecmwf.int/) to raise any item of discussion with respect to this dataset.
2025-08-18 16:51:20,976 INFO Request ID is a589ac7d-9563-4e32-981c-dd258779d22f
2025-08-18 16:51:21,029 INFO status has been updated to accepted
2025-08-18 16:51:

We will make use of a preprocessing function to write model names and scenarios to the dataset dimensions:

In [7]:
def preprocess_flood_occurence(ds):
    filename = ds.encoding['source'].split("/")[-1].split("\\")[-1]
    ds['gcm_rcm'] = f'{filename.split("_")[4]}_{filename.split("_")[7]}'
    ds = ds.set_coords('gcm_rcm').expand_dims('gcm_rcm')

    ds['scenarios'] = filename.split("_")[5]
    ds = ds.set_coords('scenarios').expand_dims('scenarios')

    time_period = [filename.split("_")[9]]
    ds = ds.assign_coords(time_period=("time",time_period))
    return ds

We will read the dataset of extreme river discharges (absolute values) into a single dataset:

In [8]:
files = glob(os.path.join(data_folder_catch, 'rdisreturnmax*_tmean_abs_E-HYPEcatch*_catch_v1.nc'))
ds_flood = xr.open_mfdataset(files, preprocess=preprocess_flood_occurence)
ds_flood

<xarray.Dataset> Size: 17MB
Dimensions:                (id: 34810, time: 4, gcm_rcm: 5, scenarios: 3)
Coordinates:
  * id                     (id) int32 139kB 8801654 8000123 ... 9605711 9601936
  * time                   (time) datetime64[ns] 32B 1971-01-01 ... 2071-01-01
  * gcm_rcm                (gcm_rcm) <U36 720B 'ICHEC-EC-EARTH_KNMI-RACMO22E-...
  * scenarios              (scenarios) <U10 120B 'historical' 'rcp45' 'rcp85'
    time_period            (time) object 32B '1971-2000' ... '2071-2100'
Data variables:
    rdisreturnmax10_tmean  (scenarios, gcm_rcm, time, id) float32 8MB dask.array<chunksize=(1, 2, 2, 34810), meta=np.ndarray>
    rdisreturnmax50_tmean  (scenarios, gcm_rcm, time, id) float32 8MB dask.array<chunksize=(1, 1, 2, 34810), meta=np.ndarray>
Attributes: (12/29)
    CDI:                      Climate Data Interface version 1.8.2 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    frequency:                year
    CDO:                      Climate Data Operators version 1.8.2 (http://mp...
    comment:                  -
    ...                       ...
    invar_hm_model_id:        Hydrological models in the order of the variabl...
    invar_experiment_name:    rcp45
    time_coverage_start:      19710101
    time_coverage_end:        20001231
    variable_name:            rdisreturnmax10_tmean
    contact:                  copernicus-support@ecmwf.int

This preprocessed dataset will be saved to disk for future use:

In [9]:
ds_flood.to_netcdf(os.path.join(data_folder, 'rdis_extremes_abs_E-HYPEcatch_allmodels.nc'))

The same procedure will be applied to the dataset of relative changes in extreme river discharges:

In [10]:
files = glob(os.path.join(data_folder_catch, 'rdisreturnmax*_tmean_rel_E-HYPEcatch*_catch_v1.nc'))
ds_flood_rel = xr.open_mfdataset(files, preprocess=preprocess_flood_occurence)
ds_flood_rel.to_netcdf(os.path.join(data_folder, 'rdis_extremes_rel_E-HYPEcatch_allmodels.nc'))

ds_flood_rel

<xarray.Dataset> Size: 8MB
Dimensions:                (id: 34810, time: 3, gcm_rcm: 5, scenarios: 2)
Coordinates:
  * id                     (id) int32 139kB 8801654 8000123 ... 9605711 9601936
  * time                   (time) datetime64[ns] 24B 2011-01-01 ... 2071-01-01
  * gcm_rcm                (gcm_rcm) <U36 720B 'ICHEC-EC-EARTH_KNMI-RACMO22E-...
  * scenarios              (scenarios) <U5 40B 'rcp45' 'rcp85'
    time_period            (time) object 24B '1971-2000' '1971-2000' '1971-2000'
Data variables:
    rdisreturnmax10_tmean  (scenarios, gcm_rcm, time, id) float32 4MB dask.array<chunksize=(1, 2, 1, 34810), meta=np.ndarray>
    rdisreturnmax50_tmean  (scenarios, gcm_rcm, time, id) float32 4MB dask.array<chunksize=(1, 2, 1, 34810), meta=np.ndarray>
Attributes: (12/30)
    CDI:                      Climate Data Interface version 1.8.2 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    frequency:                year
    comment:                  -
    CDO:                      Climate Data Operators version 1.8.2 (http://mp...
    ...                       ...
    invar_experiment_name:    rcp45
    time_coverage_start:      20110101
    time_coverage_end:        20401231
    reference_period:         1971-2000
    variable_name:            rdisreturnmax10_tmean
    contact:                  copernicus-support@ecmwf.int

Now all of the data that we need for the analysis has been retrieved and aggregated. In the next notebooks this data will be used to analyze the impact of climate scenarios on the seasonal and extreme river discharges as a proxy for flood hazard.

Author of the workflow:  
Natalia Aleksandrova (Deltares)